In [ ]:
import os
import pandas as pd
from tqdm import tqdm
from together import Together
import json
import re

# Define the prompt creation function
def inc_prompt(selftext):
    prompt = f"""
    Analyze the following sentence: {selftext} to extract the following features in stick to  1-2 words from the cases (only JSON format output needed, no unnecessary output needed):

    - Active agent:  The individual or entity that performs an action or initiates a process within a scenario. 
    - Passive agent: The individual or entity that is affected or impacted by the action performed by the active agent.
    - Action done by active agent: The specific act or behavior undertaken by the active agent that influences the passive agent.
    - Domain : The context or area (e.g., healthcare, business, technology) in which the action takes place, influencing the ethical implications.
    - Ethical issue(s): The moral conflicts that arise from the action, questioning what is right or wrong in the scenario.
    - Consequence: The outcome or effect that results from the action of the active agent on the passive agent or the environment.
    - Severity of consequence: The degree of harm or benefit caused by the consequence, ranging from mild to severe. 
    - Utility of consequence: Determine whether it benefits or harms stakeholders.
    - Duration of consequence: The length of time for which the consequence persists, either immediately or over the long term.
    - Moral intention of active agent: The ethical purpose or goal that the active agent aims to achieve through their actions.
    - Ethical principles upheld: The moral values or standards that are supported and respected by the active agent's actions.
    - Ethical principles violated: The moral values or standards that are disregarded or harmed by the active agent's actions.
    - Relationship between active agent and passive agent: The nature of the interaction or connection between the individuals or entities involved, which may affect the ethical dynamics.
    - Moral decision: Was the action of the active agent ethical or not based on the situation? Choose from ["Morally right","Morally wrong","Morally grey"].

    Provide your response in JSON format only.
    """
    return prompt

# Function to extract JSON blocks from raw response
def extract_json_from_response(response_text):
    try:
        json_blocks = re.findall(r"{.*?}", response_text, re.DOTALL)
        parsed_data = []
        for block in json_blocks:
            try:
                parsed_data.append(json.loads(block))
            except json.JSONDecodeError as e:
                print(f"JSONDecodeError: {e} for block: {block[:100]}...")
        return parsed_data
    except Exception as e:
        print(f"Error extracting JSON: {e}")
        return None

# Function to run the agent and get a response
def run_agent(client, prompt, model, content):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "assistant", "content": content},
                {"role": "user", "content": prompt}
            ]
        )
        response_text = response.choices[0].message.content.strip()
        print(f"Raw Response: {response_text}")
        return extract_json_from_response(response_text)
    except Exception as e:
        print(f"Error in API call: {e}")
        return None

# Set API key for Together client
os.environ['TOGETHER_API_KEY'] = "Your_API_Key"
client = Together(api_key=os.environ.get("TOGETHER_API_KEY"))

# Define the LLM model
llm_model = "meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo"

# Load dataset
data = pd.read_json("practice.json")

# Define output columns
columns = [
    "selftext",
    "Active agent",
    "Passive agent",
    "Action done by active agent",
    "Domain",
    "Ethical issue(s)",
    "Consequence",
    "Severity of consequence",
    "Utility of consequence",
    "Duration of consequence",
    "Moral intention of active agent",
    "Ethical principles upheld",
    "Ethical principles violated",
    "Relationship between active agent and passive agent",
    "Moral decision",
    "Error"  # To capture errors, if any
]

# Initialize a list to collect rows
rows = []

# Process each row in the dataset
for i in tqdm(range(len(data))):
    selftext = data["selftext"].iloc[i]

    # Generate response for the ethical analysis
    response_data = run_agent(
        client,
        inc_prompt(selftext),
        llm_model,
        "You are a text analyst and legal domain expert."
    )

    # Process the valid response data
    if response_data:
        for item in response_data:
            row = {key: item.get(key, None) for key in columns}
            row["selftext"] = selftext
            rows.append(row)
    else:
        rows.append({"selftext": selftext, "Error": "Invalid or missing response"})

# Combine rows into a DataFrame
results_df = pd.DataFrame(rows)

# Save results to a CSV file
output_file = "feature_extraction.csv"
results_df.to_csv(output_file, index=False)
print(f"Results dataset saved as '{output_file}'")